In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

# initial tumor volume = 1mm^3

# set up input box

lambda1 = 0.192 #[1/day]
b = 5.85 #[1/day]
d = 0.00873 #[1/day*mm^2]
e = 0.66 #[1/day*mg]
u(t) = #angiogenic inhibitor effect
tstep = #yo mama
epi = 0.000001
max_error = 10

dx1dt(t) = -lambda1*x1(t)*log(x1(t)/x2(t))
dx2dt(t) = b*x1(t)-d*(x1(t))^(2/3)*x2(t)-e*x2(t)*u(t)

x1 = []
x2 = []

while max_error>epi:
    icount += 1
    for j in range (1,n1):
        y[j] = -(1/2)*(c[j]-(u[j+1]+u[j-1]))
        err = np.abs(y[j] - u[j])
        L_err.append(err)
        u[j] = y[j] 
    max_error = max(L_err)    
    L_err = []